# Piano Bot notebook
This notebook was made for training the models on Google Colab. It works by mounting the personal google drive of the user, selecting a folder (in the fashion of a workspace) and cloning the github repo there. All the code is in the python files, so in this notebook there's just a sequence of commands to get it all up and running. 

The graphics for the training and evaluation loss are displayed on tensorboard. The easiest way to access tensorboard is on the local PC, since the default behaviour is to open a server on localhost that can be accessed on the web browser. It is advised then to download the logs folder when the training is done, and run the tensorboard command in that folder to create the tensorboard server.


In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
! ls

gdrive	sample_data


Specify your drive path here:

In [6]:
%cd gdrive/My Drive/Universidad/DTU/Deep_Learning/Project/

/content/gdrive/My Drive/Universidad/DTU/Deep_Learning/Project


Clone github (with the data):

In [ ]:
! git clone https://github.com/pearlww/PianoBot.git

fatal: destination path 'PianoBot' already exists and is not an empty directory.


Required libraries:

In [7]:
%cd PianoBot/
! pip install tensorboardX
! pip install pretty_midi
! pip install progress
! pip install timidity

/content/gdrive/MyDrive/Universidad/DTU/Deep_Learning/Project/PianoBot


All parameters are set in config.py.
We developed two main models: the basic transformer from "Attention is all you need" paper and the music transformer from the paper with the same name.

In [8]:
# import our defined files
import config
from music_transformer import MusicTransformer
from transformer import Transformer
import loss_functions
from data_loader import DataLoader
import utils

import datetime
import torch
import torch.optim as optim
from tensorboardX import SummaryWriter


In [9]:
# check cuda
if torch.cuda.is_available():
    config.device = torch.device('cuda')
else:
    config.device = torch.device('cpu')
print('| Summary - Device Info : {}'.format(torch.cuda.device))

| Summary - Device Info : <class 'torch.cuda.device'>


In [10]:
#define model as music transformer or normal transformer
model = MusicTransformer(
            embedding_dim=config.embedding_dim,
            vocab_size=config.vocab_size,
            num_layer=config.num_layers,
            max_seq= config.max_seq,
            dropout=config.dropout,
)

In [ ]:
model = Transformer(
            embedding_dim=config.embedding_dim,
            vocab_size=config.vocab_size,
            num_layer=config.num_layers,
            max_seq= config.max_seq,
            dropout=config.dropout,
)

In [11]:
model.to(config.device)
print(model)

MusicTransformer(
  (Encoder): EncoderMusic(
    (embedding): Embedding(391, 256, padding_idx=388)
    (pos_encoding): DynamicPositionEmbedding()
    (enc_layers): ModuleList(
      (0): EncoderMusicLayer(
        (rga): RelativeGlobalAttention(
          (Wq): Linear(in_features=256, out_features=256, bias=True)
          (Wk): Linear(in_features=256, out_features=256, bias=True)
          (Wv): Linear(in_features=256, out_features=256, bias=True)
          (fc): Linear(in_features=256, out_features=256, bias=True)
        )
        (FFN_pre): Linear(in_features=256, out_features=128, bias=True)
        (FFN_suf): Linear(in_features=128, out_features=256, bias=True)
        (layernorm1): LayerNorm((256,), eps=1e-06, elementwise_affine=True)
        (layernorm2): LayerNorm((256,), eps=1e-06, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): EncoderMusicLayer(
        (rga): RelativeGlobalAtten

In [12]:
# load data
dataset = DataLoader(config.pickle_dir+"high/", config.pickle_dir+"low/")
# define optimizer and criterion
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)
criterion = loss_functions.TransformerLoss()


# define tensorboard writer
current_time = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
train_log_dir = 'logs/'+config.experiment+'/'+current_time+'/train'
eval_log_dir = 'logs/'+config.experiment+'/'+current_time+'/eval'
train_summary_writer = SummaryWriter(train_log_dir)
eval_summary_writer = SummaryWriter(eval_log_dir)

In [13]:
# Train Start
print(">> Train start...")
idx = 0
for e in range(config.epochs):
    print(">>> [Epoch was updated]")
    for b in range(len(dataset.X) // config.batch_size):
        model.train()
        batch_x, batch_y = dataset.batch(config.batch_size, config.max_seq, 'train') 

        # l = max_seq
        batch_x = torch.from_numpy(batch_x).contiguous().to(config.device, non_blocking=True, dtype=torch.int)
        # l = max_seq+2
        batch_y = torch.from_numpy(batch_y).contiguous().to(config.device, non_blocking=True, dtype=torch.int)

        # right shifted,  l = max_seq +1
        target_inputs = batch_y[:, :-1]
        targets = batch_y[:, 1:]

        preds = model.forward(batch_x, target_inputs)

        loss = criterion(preds, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_summary_writer.add_scalar('loss', loss, global_step=idx)
        torch.cuda.empty_cache()
        idx += 1

        if b % 100 ==0:
            model.eval()
            eval_x, eval_y = dataset.batch(2, config.max_seq, 'eval')
            eval_x = torch.from_numpy(eval_x).contiguous().to(config.device, dtype=torch.int)
            eval_y = torch.from_numpy(eval_y).contiguous().to(config.device, dtype=torch.int)

            eval_target_inputs = eval_y[:, :-1]
            eval_targets = eval_y[:, 1:]

            eval_preds = model.forward(eval_x, eval_target_inputs)
            eval_loss = criterion(eval_preds, eval_targets)
            eval_summary_writer.add_scalar('loss', eval_loss, global_step=idx)

            print('\n====================================================')
            print('Epoch:{}/{}'.format(e, config.epochs))
            # print('Batch: {}/{}'.format(b, len(dataset.X) // config.batch_size))
            print('Train >>>> Loss: {:6.6}'.format(loss))
            print('Eval >>>> Loss: {:6.6}'.format(eval_loss))

    if e%10 == 0:
        torch.save(model.state_dict(), config.model_dir+'/train-{}.pth'.format(e))


torch.save(model.state_dict(), config.model_dir+'/final.pth'.format(idx))
eval_summary_writer.close()
train_summary_writer.close()

>> Train start...
>>> [Epoch was updated]

Epoch:0/100
Train >>>> Loss: 6.14243
Eval >>>> Loss: 6.12814


KeyboardInterrupt: ignored

To get the training graph it's better to run it on local, so we can access tensorboard easily:

In [ ]:
%cd logs

[Errno 2] No such file or directory: 'logs'
/content


In [1]:
! tensorboard --logdir .

2021-01-04 21:50:29.756683: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.4.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


In [ ]:
%cd ..

/content/gdrive/MyDrive/Universidad/DTU/Deep_Learning


To generate a test sequence given the high notes of a random one:

In [ ]:
! python generate.py

inputs: [[355 264 371  71 275 199 369  71 305 375  72 256 199 256 370  67 273 373
   74 260 200 375  72 261 202 368  74 260 200 371  72 257 195 370  67 271
  370  66 274 373  71 259 364  64 258 200 367  72 264 202 373  74 295 194
  371  66 272 370  67 271 195 199 192 200 194 259 202 256 372  71 272 375
   74 273 375  78 265 199 202 264 374  79 258 206 270 375  76 268 207 259
  204 256 373  79 272 373  74 269 207 257 202 256 371  79 275 373  72 273
  374  71 274 200 374  72 256 371  66 260 195 266 374  81 274 200 373  72
  273 199]]
targets: [[355 283 367  55 273 369  59 258 183 269 369  62 290 371  57 328 187 368
   59 273 190 371  62 307 185 190 256 187 294 370  59 271 187 277 369  60
  270 188 274 370  59 273 187 276 367  57 326 372  55 293 371  59 292 370
   62 272 370  54 273 368  52 270 371  50 277 370  49 292 180 372  52 291
  185 368  57 292 177 371  49 292 178 372  50 293 182 370  54 271 190 373
   62 275 185 370  57 290 371  48 266 183 187 180 177 178 182 190 257 176
  281 369

The output midi is in /output/generated.mid

---Some debug code---

In [ ]:
i=0
cut=[]
while i<len(seq) and seq[i] != 390:
    cut.append(seq[i])
    i+=1
print(cut)
from processor import decode_midi
import config
decode_midi(cut, file_path=config.save_path)

[261, 291, 272, 295, 62, 294, 283, 43, 60, 190, 55, 190, 61, 62, 190, 171, 280, 54, 300, 58, 59, 266, 55, 62, 31, 292, 31, 183, 256, 257]


In [ ]:
%cd Project

/content/gdrive/My Drive/Universidad/DTU/Deep_Learning/Project
